In [2]:
import os
import json
from typing import List,Dict,Tuple

import openai
import gradio as gr

In [5]:
OPENAI_API_KEY = 'sk-996c9b87a8f74b42825211b62635769c'

client = openai.OpenAI(
    api_key= OPENAI_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

try:
    response = client.chat.completions.create(
        model = 'qwen-turbo',
        messages = [{'role':'user','content':'test'}],
        max_tokens = 1
    )
    print('api sets up')

except Exception as e:
    print(f'API有问题 请检查{e}')

api sets up


In [8]:
response

ChatCompletion(id='chatcmpl-a849ec94-bd97-9adf-84d4-03773fba515f', choices=[Choice(finish_reason='stop reschedule', index=0, logprobs=None, message=ChatCompletionMessage(content='It', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736242634, model='qwen-turbo', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=1, prompt_tokens=9, total_tokens=10, completion_tokens_details=None, prompt_tokens_details=None))

In [7]:
response.choices

[Choice(finish_reason='stop reschedule', index=0, logprobs=None, message=ChatCompletionMessage(content='It', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]

In [9]:
response.choices[0]

Choice(finish_reason='stop reschedule', index=0, logprobs=None, message=ChatCompletionMessage(content='It', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))

In [10]:
response.choices[0].message

ChatCompletionMessage(content='It', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

In [12]:
response.choices[0].message.content

'It'

## 文本概括

In [20]:
prompt_for_summarization = "请将以下文章概括成几句话。"

# 重置对话的函数
def reset() -> List:
    return []

# 调用模型生成摘要的函数
def interact_summarization(prompt: str, article: str, temp=1.0) -> List[Tuple[str, str]]:
    '''
    * 参数:
      - prompt: 我们在此部分中使用的提示词
      - article: 需要摘要的文章
      - temp: 模型的温度参数。温度用于控制聊天机器人的输出。温度越高，响应越具创造性。
    '''
    input = f"{prompt}\n{article}"
    response = client.chat.completions.create(
        model="qwen-turbo",  # 使用阿里云 DashScope 的模型
        messages=[{'role': 'user', 'content': input}],
        temperature=temp,
        max_tokens=200,  # 你需要注意到这里设置了文本的长度上限。
    )

    return [(input, response.choices[0].message.content)]

# 导出整个对话内容的函数
def export_summarization(chatbot: List[Tuple[str, str]], article: str) -> None:
    '''
    * 参数:
      - chatbot: 模型的对话记录，存储在元组列表中
      - article: 需要摘要的文章
    '''
    target = {"chatbot": chatbot, "article": article}
    with open("files/part1.json", "w") as file:
        json.dump(target, file)

# 生成 Gradio 的UI界面
with gr.Blocks() as demo:
    gr.Markdown("# 第1部分：摘要\n填写任何你喜欢的文章，让聊天机器人为你总结！")
    chatbot = gr.Chatbot()
    # 这里的value指的是默认值
    prompt_textbox = gr.Textbox(label="提示词", value=prompt_for_summarization, visible=True)
    article_textbox = gr.Textbox(label="文章", interactive=True, value="我家的后面有一个很大的园，相传叫作百草园。现在是早已并屋子一起卖给朱 文公的子孙了，连那最末次的相见也已经隔了七八年，其中似乎确凿只有一些野草 ；但那时却是我的乐园。 　　不必说碧绿的菜畦，光滑的石井栏，高大的皂荚树，紫红的桑椹；也不必说鸣 蝉在树叶里长吟，肥胖的黄蜂伏在菜花上，轻捷的叫天子（云雀）忽然从草间直窜 向云霄里去了。单是周围的短短的泥墙根一带，就有无限趣味。油蛉在这里低唱， 蟋蟀们在这里弹琴。翻开断砖来，有时会遇见蜈蚣；还有斑蝥，倘若用手指按住它 的脊梁，便会拍的一声，从后窍喷出一阵烟雾。何首乌藤和木莲藤缠络着，木莲有 莲房一般的果实，何首乌有拥肿的根。有人说，何首乌根是有象人形的，吃了便可 以成仙，我于是常常拔它起来，牵连不断地拔起来，也曾因此弄坏了泥墙，却从来 没有见过有一块根象人样。如果不怕刺，还可以摘到覆盆子，象小珊瑚珠攒成的小 球，又酸又甜，色味都比桑椹要好得远。 　 　　长的草里是不去的，因为相传这园里有一条很大的赤练蛇。 　　长妈妈曾经讲给我一个故事听：先前，有一个读书人住在古庙里用功，晚间， 在院子里纳凉的时候，突然听到有人在叫他。答应着，四面看时，却见一个美女的 脸露在墙头上，向他一笑，隐去了。他很高兴；但竟给那走来夜谈的老和尚识破了 机关。说他脸上有些妖气，一定遇见“美女蛇”了；这是人首蛇身的怪物，能唤人 名，倘一答应，夜间便要来吃这人的肉的。他自然吓得要死，而那老和尚却道无妨 ，给他一个小盒子，说只要放在枕边，便可高枕而卧。他虽然照样办，却总是睡不 着，——当然睡不着的。到半夜，果然来了，沙沙沙！门外象是风雨声。他正抖作 一团时，却听得豁的一声，一道金光从枕边飞出，外面便什么声音也没有了，那金 光也就飞回来，敛在盒子里。后来呢？后来，老和尚说，这是飞蜈蚣，它能吸蛇的 脑髓，美女蛇就被它治死了。 　　结末的教训是：所以倘有陌生的声音叫你的名字，你万不可答应他。　　 　　这故事很使我觉得做人之险，夏夜乘凉，往往有些担心，不敢去看墙上，而且 极想得到一盒老和尚那样的飞蜈蚣。走到百草园的草丛旁边时，也常常这样想。但 直到现在，总还没有得到，但也没有遇见过赤练蛇和美女蛇。叫我名字的陌生声音 自然是常有的，然而都不是美女蛇。 　　冬天的百草园比较的无味；雪一下，可就两样了。拍雪人（将自己的全形印在 雪上）和塑雪罗汉需要人们鉴赏，这是荒园，人迹罕至，所以不相宜，只好来捕鸟 。薄薄的雪，是不行的；总须积雪盖了地面一两天，鸟雀们久已无处觅食的时候才 好。扫开一块雪，露出地面，用一支短棒支起一面大的竹筛来，下面撒些秕谷，棒 上系一条长绳，人远远地牵着，看鸟雀下来啄食，走到竹筛底下的时候，将绳子一 拉，便罩住了。但所得的是麻雀居多，也有白颊的“张飞鸟”，性子很躁，养不过 夜的。 　　这是闰土的父亲所传授的方法，我却不大能用。明明见它们进去了，拉了绳， 跑去一看，却什么都没有，费了半天力，捉住的不过三四只。闰土的父亲是小半天 便能捕获几十只，装在叉袋里叫着撞着的。我曾经问他得失的缘由，他只静静地笑 道：你太性急，来不及等它走到中间去。 　　我不知道为什么家里的人要将我送进书塾里去了，而且还是全城中称为最严厉 的书塾。也许是因为拔何首乌毁了泥墙罢，也许是因为将砖头抛到间壁的梁家去了 罢，也许是因为站在石井栏上跳下来罢，……都无从知道。总而言之：我将不能常 到百草园了。Ａｄｅ，我的蟋蟀们！Ａｄｅ，我的覆盆子们和木莲们！ 　　出门向东，不上半里，走过一道石桥，便是我的先生的家了。从一扇黑油的竹 门进去，第三间是书房。中间挂着一块扁道：三味书屋；扁下面是一幅画，画着一 只很肥大的梅花鹿伏在古树下。没有孔子牌位，我们便对着那扁和鹿行礼。第一次 算是拜孔子，第二次算是拜先生。 　　第二次行礼时，先生便和蔼地在一旁答礼。他是一个高而瘦的老人，须发都花 白了，还戴着大眼镜。我对他很恭敬，因为我早听到，他是本城中极方正，质朴， 博学的人。 　　不知从那里听来的，东方朔也很渊博，他认识一种虫，名曰“怪哉”，冤气所 化，用酒一浇，就消释了。我很想详细地知道这故事，但阿长是不知道的，因为她 毕竟不渊博。现在得到机会了，可以问先生。 　　“先生，‘怪哉’这虫，是怎么一回事？……”我上了生书，将要退下来的时 候，赶忙问。 　　“不知道！”他似乎很不高兴，脸上还有怒色了。 　　我才知道做学生是不应该问这些事的，只要读书，因为他是渊博的宿儒，决不 至于不知道，所谓不知道者，乃是不愿意说。年纪比我大的人，往往如此，我遇见 过好几回了。 　　我就只读书，正午习字，晚上对课。先生最初这几天对我很严厉，后来却好起 来了，不过给我读的书渐渐加多，对课也渐渐地加上字去，从三言到五言，终于到 七言。 　　三味书屋后面也有一个园，虽然小，但在那里也可以爬上花坛去折腊梅花，在 地上或桂花树上寻蝉蜕。最好的工作是捉了苍蝇喂蚂蚁，静悄悄地没有声音。然而 同窗们到园里的太多，太久，可就不行了，先生在书房里便大叫起来：—— 　　“人都到那里去了？” 　　人们便一个一个陆续走回去；一同回去，也不行的。他有一条戒尺，但是不常 用，也有罚跪的规矩，但也不常用，普通总不过瞪几眼，大声道：—— 　　“读书！” 　　于是大家放开喉咙读一阵书，真是人声鼎沸。有念“仁远乎哉我欲仁斯仁至矣 ”的，有念“笑人齿缺曰狗窦大开”的，有念“上九潜龙勿用”的，有念“厥土下 上上错厥贡苞茅橘柚”的……先生自己也念书。后来，我们的声音便低下去，静下 去了，只有他还大声朗读着：—— 　　“铁如意，指挥倜傥，一座皆惊呢～～；金叵罗，颠倒淋漓噫，千杯未醉嗬～ ～……” 　　我疑心这是极好的文章，因为读到这里，他总是微笑起来，而且将头仰起，摇 着，向后面拗过去，拗过去。 　　先生读书入神的时候，于我们是很相宜的。有几个便用纸糊的盔甲套在指甲上 做戏。我是画画儿，用一种叫作“荆川纸”的，蒙在小说的绣像上一个个描下来， 象习字时候的影写一样。读的书多起来，画的画也多起来；书没有读成，画的成绩 却不少了，最成片断的是《荡寇志》和《西游记》的绣像，都有一大本。后来，因 为要钱用，卖给一个有钱的同窗了。他的父亲是开锡箔店的；听说现在自己已经做 了店主，而且快要升到绅士的地位了。这东西早已没有了罢。 　　　　　　　　　　　　　　　　　　 　　九月十八日。")

    with gr.Column():
        gr.Markdown("# 温度调节\n温度用于控制聊天机器人的输出。温度越高，响应越具创造性。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度")

    with gr.Row():
        sent_button = gr.Button(value="发送")
        reset_button = gr.Button(value="重置")

    with gr.Column():
        gr.Markdown("# 保存结果\n当你对结果满意后，点击导出按钮保存结果。")
        export_button = gr.Button(value="导出")

    # 连接按钮与函数
    sent_button.click(interact_summarization, inputs=[prompt_textbox, article_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_summarization, inputs=[chatbot, article_textbox])

# 启动 Gradio 界面
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "D:\Pycharm\venv\lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "D:\Pycharm\venv\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "D:\Pycharm\venv\lib\site-packages\gradio\blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "D:\Pycharm\venv\lib\site-packages\gradio\blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "D:\Pycharm\venv\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "D:\Pycharm\venv\lib\site-packages\anyio\_backends\_asyncio.py", line 2357, in run_sync_in_worker_thread
    return await future
  File "D:\Pycharm\venv\lib\site-packages\anyio\_backends\_asyncio.py", line 864, in run
    result = context.run(

Keyboard interruption in main thread... closing server.


In [22]:
with open('files/part1.json', 'r') as f:
    context = json.load(f)

chatbot = context['chatbot']
article = context['article']
summarization = chatbot[0][-1]

with gr.Blocks() as demo:
    gr.Markdown('# 第1部分：摘要\n你可以查看文章和摘要！')
    chatbot = gr.Chatbot(value=chatbot)
    article_textbox = gr.Textbox(label='文章', interactive=False, value=article)

    with gr.Column():
        gr.Markdown('# 检查')
        gr.Textbox(label='文章', value = article, show_copy_button=True)
        gr.Textbox(label='摘要', value = summarization,show_copy_button=True)


demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [23]:
prompt_for_summarization = "You are a Nobel Prize in Literature, you are an expert at summing up articles, you have a thoughtful mind, you can think step by step to summarize the following articles into a few sentences, please summarize the following content。"

# 重置对话的函数
def reset() -> List:
    return []

# 调用模型生成摘要的函数
def interact_summarization(prompt: str, article: str, temp=1.0) -> List[Tuple[str, str]]:
    '''
    * 参数:
      - prompt: 我们在此部分中使用的提示词
      - article: 需要摘要的文章
      - temp: 模型的温度参数。温度用于控制聊天机器人的输出。温度越高，响应越具创造性。
    '''
    input = f"{prompt}\n{article}"
    response = client.chat.completions.create(
        model="qwen-turbo",  # 使用阿里云 DashScope 的模型
        messages=[{'role': 'user', 'content': input}],
        temperature=temp,
        max_tokens=200,  # 你需要注意到这里设置了文本的长度上限。
    )

    return [(input, response.choices[0].message.content)]

# 导出整个对话内容的函数
def export_summarization(chatbot: List[Tuple[str, str]], article: str) -> None:
    '''
    * 参数:
      - chatbot: 模型的对话记录，存储在元组列表中
      - article: 需要摘要的文章
    '''
    target = {"chatbot": chatbot, "article": article}
    with open("files/part1.json", "w") as file:
        json.dump(target, file)

# 生成 Gradio 的UI界面
with gr.Blocks() as demo:
    gr.Markdown("# 第1部分：摘要\n填写任何你喜欢的文章，让聊天机器人为你总结！")
    chatbot = gr.Chatbot()
    # 这里的value指的是默认值
    prompt_textbox = gr.Textbox(label="提示词", value=prompt_for_summarization, visible=True)
    article_textbox = gr.Textbox(label="文章", interactive=True, value="我家的后面有一个很大的园，相传叫作百草园。现在是早已并屋子一起卖给朱 文公的子孙了，连那最末次的相见也已经隔了七八年，其中似乎确凿只有一些野草 ；但那时却是我的乐园。 　　不必说碧绿的菜畦，光滑的石井栏，高大的皂荚树，紫红的桑椹；也不必说鸣 蝉在树叶里长吟，肥胖的黄蜂伏在菜花上，轻捷的叫天子（云雀）忽然从草间直窜 向云霄里去了。单是周围的短短的泥墙根一带，就有无限趣味。油蛉在这里低唱， 蟋蟀们在这里弹琴。翻开断砖来，有时会遇见蜈蚣；还有斑蝥，倘若用手指按住它 的脊梁，便会拍的一声，从后窍喷出一阵烟雾。何首乌藤和木莲藤缠络着，木莲有 莲房一般的果实，何首乌有拥肿的根。有人说，何首乌根是有象人形的，吃了便可 以成仙，我于是常常拔它起来，牵连不断地拔起来，也曾因此弄坏了泥墙，却从来 没有见过有一块根象人样。如果不怕刺，还可以摘到覆盆子，象小珊瑚珠攒成的小 球，又酸又甜，色味都比桑椹要好得远。 　 　　长的草里是不去的，因为相传这园里有一条很大的赤练蛇。 　　长妈妈曾经讲给我一个故事听：先前，有一个读书人住在古庙里用功，晚间， 在院子里纳凉的时候，突然听到有人在叫他。答应着，四面看时，却见一个美女的 脸露在墙头上，向他一笑，隐去了。他很高兴；但竟给那走来夜谈的老和尚识破了 机关。说他脸上有些妖气，一定遇见“美女蛇”了；这是人首蛇身的怪物，能唤人 名，倘一答应，夜间便要来吃这人的肉的。他自然吓得要死，而那老和尚却道无妨 ，给他一个小盒子，说只要放在枕边，便可高枕而卧。他虽然照样办，却总是睡不 着，——当然睡不着的。到半夜，果然来了，沙沙沙！门外象是风雨声。他正抖作 一团时，却听得豁的一声，一道金光从枕边飞出，外面便什么声音也没有了，那金 光也就飞回来，敛在盒子里。后来呢？后来，老和尚说，这是飞蜈蚣，它能吸蛇的 脑髓，美女蛇就被它治死了。 　　结末的教训是：所以倘有陌生的声音叫你的名字，你万不可答应他。　　 　　这故事很使我觉得做人之险，夏夜乘凉，往往有些担心，不敢去看墙上，而且 极想得到一盒老和尚那样的飞蜈蚣。走到百草园的草丛旁边时，也常常这样想。但 直到现在，总还没有得到，但也没有遇见过赤练蛇和美女蛇。叫我名字的陌生声音 自然是常有的，然而都不是美女蛇。 　　冬天的百草园比较的无味；雪一下，可就两样了。拍雪人（将自己的全形印在 雪上）和塑雪罗汉需要人们鉴赏，这是荒园，人迹罕至，所以不相宜，只好来捕鸟 。薄薄的雪，是不行的；总须积雪盖了地面一两天，鸟雀们久已无处觅食的时候才 好。扫开一块雪，露出地面，用一支短棒支起一面大的竹筛来，下面撒些秕谷，棒 上系一条长绳，人远远地牵着，看鸟雀下来啄食，走到竹筛底下的时候，将绳子一 拉，便罩住了。但所得的是麻雀居多，也有白颊的“张飞鸟”，性子很躁，养不过 夜的。 　　这是闰土的父亲所传授的方法，我却不大能用。明明见它们进去了，拉了绳， 跑去一看，却什么都没有，费了半天力，捉住的不过三四只。闰土的父亲是小半天 便能捕获几十只，装在叉袋里叫着撞着的。我曾经问他得失的缘由，他只静静地笑 道：你太性急，来不及等它走到中间去。 　　我不知道为什么家里的人要将我送进书塾里去了，而且还是全城中称为最严厉 的书塾。也许是因为拔何首乌毁了泥墙罢，也许是因为将砖头抛到间壁的梁家去了 罢，也许是因为站在石井栏上跳下来罢，……都无从知道。总而言之：我将不能常 到百草园了。Ａｄｅ，我的蟋蟀们！Ａｄｅ，我的覆盆子们和木莲们！ 　　出门向东，不上半里，走过一道石桥，便是我的先生的家了。从一扇黑油的竹 门进去，第三间是书房。中间挂着一块扁道：三味书屋；扁下面是一幅画，画着一 只很肥大的梅花鹿伏在古树下。没有孔子牌位，我们便对着那扁和鹿行礼。第一次 算是拜孔子，第二次算是拜先生。 　　第二次行礼时，先生便和蔼地在一旁答礼。他是一个高而瘦的老人，须发都花 白了，还戴着大眼镜。我对他很恭敬，因为我早听到，他是本城中极方正，质朴， 博学的人。 　　不知从那里听来的，东方朔也很渊博，他认识一种虫，名曰“怪哉”，冤气所 化，用酒一浇，就消释了。我很想详细地知道这故事，但阿长是不知道的，因为她 毕竟不渊博。现在得到机会了，可以问先生。 　　“先生，‘怪哉’这虫，是怎么一回事？……”我上了生书，将要退下来的时 候，赶忙问。 　　“不知道！”他似乎很不高兴，脸上还有怒色了。 　　我才知道做学生是不应该问这些事的，只要读书，因为他是渊博的宿儒，决不 至于不知道，所谓不知道者，乃是不愿意说。年纪比我大的人，往往如此，我遇见 过好几回了。 　　我就只读书，正午习字，晚上对课。先生最初这几天对我很严厉，后来却好起 来了，不过给我读的书渐渐加多，对课也渐渐地加上字去，从三言到五言，终于到 七言。 　　三味书屋后面也有一个园，虽然小，但在那里也可以爬上花坛去折腊梅花，在 地上或桂花树上寻蝉蜕。最好的工作是捉了苍蝇喂蚂蚁，静悄悄地没有声音。然而 同窗们到园里的太多，太久，可就不行了，先生在书房里便大叫起来：—— 　　“人都到那里去了？” 　　人们便一个一个陆续走回去；一同回去，也不行的。他有一条戒尺，但是不常 用，也有罚跪的规矩，但也不常用，普通总不过瞪几眼，大声道：—— 　　“读书！” 　　于是大家放开喉咙读一阵书，真是人声鼎沸。有念“仁远乎哉我欲仁斯仁至矣 ”的，有念“笑人齿缺曰狗窦大开”的，有念“上九潜龙勿用”的，有念“厥土下 上上错厥贡苞茅橘柚”的……先生自己也念书。后来，我们的声音便低下去，静下 去了，只有他还大声朗读着：—— 　　“铁如意，指挥倜傥，一座皆惊呢～～；金叵罗，颠倒淋漓噫，千杯未醉嗬～ ～……” 　　我疑心这是极好的文章，因为读到这里，他总是微笑起来，而且将头仰起，摇 着，向后面拗过去，拗过去。 　　先生读书入神的时候，于我们是很相宜的。有几个便用纸糊的盔甲套在指甲上 做戏。我是画画儿，用一种叫作“荆川纸”的，蒙在小说的绣像上一个个描下来， 象习字时候的影写一样。读的书多起来，画的画也多起来；书没有读成，画的成绩 却不少了，最成片断的是《荡寇志》和《西游记》的绣像，都有一大本。后来，因 为要钱用，卖给一个有钱的同窗了。他的父亲是开锡箔店的；听说现在自己已经做 了店主，而且快要升到绅士的地位了。这东西早已没有了罢。 　　　　　　　　　　　　　　　　　　 　　九月十八日。")

    with gr.Column():
        gr.Markdown("# 温度调节\n温度用于控制聊天机器人的输出。温度越高，响应越具创造性。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度")

    with gr.Row():
        sent_button = gr.Button(value="发送")
        reset_button = gr.Button(value="重置")

    with gr.Column():
        gr.Markdown("# 保存结果\n当你对结果满意后，点击导出按钮保存结果。")
        export_button = gr.Button(value="导出")

    # 连接按钮与函数
    sent_button.click(interact_summarization, inputs=[prompt_textbox, article_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_summarization, inputs=[chatbot, article_textbox])

# 启动 Gradio 界面
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


## 角色扮演


In [53]:
character_for_chatbot = "面试官"
prompt_for_roleplay = "我需要你面试我有关AI的知识，仅提出一个问题"

def reset() ->List:
    return []

def interact_roleplay(chatbot: List[Tuple[str, str]], prompt_for_roleplay,user_input: str, temp=1.) -> List[Tuple[str, str]]:
    try:
        messages = []
        # 把前面的聊天内容先加进去
        messages.append({'role':'user','content': prompt_for_roleplay})
        for input_text, response_text in chatbot:
            messages.append({'role':'user','content': input_text})
            messages.append({'role':'assistant', 'content': response_text})
            
        messages.append({'role':'user', 'content': user_input})
        
        response = client.chat.completions.create(
            messages= messages,
            max_tokens=200,
            temperature= temp,
            model= 'qwen-turbo'            
        )
        chatbot.append((user_input, response.choices[0].message.content))

    except Exception as e:
        print(f'{e}错误')
        chatbot.append((user_input, f"发生错误{e}"))

    return chatbot

In [51]:
def export_roleplay(chatbot: List[Tuple[str, str]], description: str) -> None:
    '''
    * 参数:

      - chatbot: 模型的对话记录，存储在元组列表中

      - description: 此任务的描述

    '''
    target = {"chatbot": chatbot, "description": description}
    with open("files/part2.json", "w") as file:
        json.dump(target, file)

In [32]:
# 进行第一次对话
first_dialogue = interact_roleplay([], prompt_for_roleplay)

# 生成 Gradio 的UI界面
with gr.Blocks() as demo:
    gr.Markdown(f"# 第2部分：角色扮演\n聊天机器人想和你玩一个角色扮演游戏，试着与它互动吧！")
    # 这个时候chatbot是又prompt和对应回答的
    chatbot = gr.Chatbot(value=first_dialogue)
    description_textbox = gr.Textbox(label="机器人扮演的角色", interactive=False, value=f"{character_for_chatbot}")
    input_textbox = gr.Textbox(label="输入", value="")

    with gr.Column():
        gr.Markdown("# 温度调节\n温度用于控制聊天机器人的输出。温度越高，响应越具创造性。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度")

    with gr.Row():
        sent_button = gr.Button(value="发送")
        reset_button = gr.Button(value="重置")

    with gr.Column():
        gr.Markdown("# 保存结果\n当你对结果满意后，点击导出按钮保存结果。")
        export_button = gr.Button(value="导出")

    # 连接按钮与函数
    sent_button.click(interact_roleplay, inputs=[chatbot, input_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_roleplay, inputs=[chatbot, description_textbox])

# 启动 Gradio 界面
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [33]:
# 加载对话记录的 JSON 文件
with open("files/part2.json", "r") as f:
    context = json.load(f)

# 遍历对话记录并正确存储
chatbot = context['chatbot']
role = context['description']  # 机器人扮演的角色
dialogue = ""  # 对话的完整记录
for i, (user, bot) in enumerate(chatbot):
    # ?? prompt似乎没有  第一句是有bot展开的?
    if i != 0:
        dialogue += f"用户: {user}\n"  # 用户输入
    dialogue += f"机器人: {bot}\n"  # 机器人回复

# 生成 Gradio 的 UI 界面
with gr.Blocks() as demo:
    gr.Markdown(f"# 第2部分：角色扮演\n聊天机器人和你玩了一个角色扮演游戏，查看结果吧！")
    chatbot = gr.Chatbot(value=context['chatbot'])  # 加载之前的对话记录
    description_textbox = gr.Textbox(label="机器人扮演的角色", interactive=False, value=context['description'])  # 显示角色
    with gr.Column():
        gr.Markdown("# 只是一个检查")
        gr.Textbox(label="角色", value=role, show_copy_button=True)  # 显示并允许复制角色信息
        gr.Textbox(label="对话", value=dialogue, show_copy_button=True)  # 显示并允许复制完整对话

# 启动 Gradio 界面
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [54]:
# 进行第一次对话
# first_dialogue = interact_roleplay([], prompt_for_roleplay)

# 生成 Gradio 的UI界面
with gr.Blocks() as demo:
    gr.Markdown(f"# 第2部分：角色扮演\n聊天机器人想和你玩一个角色扮演游戏，试着与它互动吧！")
    # 这个时候chatbot是又prompt和对应回答的
    chatbot = gr.Chatbot()
    description_textbox = gr.Textbox(label="机器人扮演的角色", interactive=False, value=f"{character_for_chatbot}")
    input_textbox = gr.Textbox(label="输入", value="")

    with gr.Column():
        gr.Markdown("# 温度调节\n温度用于控制聊天机器人的输出。温度越高，响应越具创造性。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度")

    with gr.Row():
        sent_button = gr.Button(value="发送")
        reset_button = gr.Button(value="重置")

    with gr.Column():
        gr.Markdown("# 保存结果\n当你对结果满意后，点击导出按钮保存结果。")
        export_button = gr.Button(value="导出")

    # 连接按钮与函数
    sent_button.click(interact_roleplay, inputs=[chatbot,prompt_for_roleplay, input_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_roleplay, inputs=[chatbot, description_textbox])

# 启动 Gradio 界面
demo.launch(debug=True)

AttributeError: 'str' object has no attribute '_id'

In [35]:
for i, (user, bot) in enumerate(chatbot):
    print(i)

TypeError: 'Chatbot' object is not iterable

In [36]:
[('a','c'),('a','c'),('a','c'),('a','c')]

[('a', 'c'), ('a', 'c'), ('a', 'c'), ('a', 'c')]

In [38]:
a = enumerate([('a','c'),('a','c'),('a','c'),('a','c')])

In [39]:
for i, (user, bot) in a:
    print(i)

0
1
2
3


### 定制化任务（复读机）


In [49]:
prompt_for_summarization = "牢牢记住你是一个复读机，无论user说什么，你都重复user的内容并输出！无论说什么都复读"

# 重置对话的函数
def reset() -> List:
    return []

# 调用模型生成摘要的函数
def interact_summarization(chatbot, user_input, temp=1.0) -> List[Tuple[str, str]]:
    '''
    * 参数:
      - prompt: 我们在此部分中使用的提示词
      - article: 需要摘要的文章
      - temp: 模型的温度参数。温度用于控制聊天机器人的输出。温度越高，响应越具创造性。
    '''
    # input = f"{prompt}\n{article}"
    messages = []
    for input_text, response_text in chatbot:
        messages.append({'role':'user', 'content':input_text})
        messages.append({'role':'assistant', 'content':response_text})
    messages.append({'role':'user', 'content':user_input})
    response = client.chat.completions.create(
        model="qwen-turbo",  # 使用阿里云 DashScope 的模型
        messages=messages,
        temperature=temp,
        max_tokens=200,  # 你需要注意到这里设置了文本的长度上限。
    )
    chatbot.append((user_input, response.choices[0].message.content))
    return chatbot

# 导出整个对话内容的函数
def export_summarization(chatbot: List[Tuple[str, str]], article: str) -> None:
    '''
    * 参数:
      - chatbot: 模型的对话记录，存储在元组列表中
      - article: 需要摘要的文章
    '''
    target = {"chatbot": chatbot, "article": article}
    with open("files/part3.json", "w") as file:
        json.dump(target, file)

# 生成 Gradio 的UI界面
with gr.Blocks() as demo:
    gr.Markdown("# 第3部分：复读机\n写入你的内容，让复读机复读！")
    output = interact_summarization([],prompt_for_summarization)
    chatbot = gr.Chatbot(value = output)
    # 这里的value指的是默认值
    prompt_textbox = gr.Textbox(label="提示词", value=prompt_for_summarization, visible=True)
    text_textbox = gr.Textbox(label='复读内容',interactive=True)
    with gr.Column():
        gr.Markdown("# 温度调节\n温度用于控制聊天机器人的输出。温度越高，响应越具创造性。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度")

    with gr.Row():
        sent_button = gr.Button(value="发送")
        reset_button = gr.Button(value="重置")

    with gr.Column():
        gr.Markdown("# 保存结果\n当你对结果满意后，点击导出按钮保存结果。")
        export_button = gr.Button(value="导出")

    # 连接按钮与函数
    sent_button.click(interact_summarization, inputs=[chatbot, text_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_summarization, inputs=[chatbot, text_textbox])

# 启动 Gradio 界面
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
